In [6]:
from llama_index.core.tools import FunctionTool
from pycoingecko import CoinGeckoAPI
from llama_index.llms.groq import Groq
from duckduckgo_search import DDGS
from typing import List, Dict, Union , Optional ,Any
import requests
import asyncio
from deepgram import DeepgramClient, SpeakOptions
import os
import json

llm = Groq(model="llama-3.1-70b-versatile", api_key="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")

In [7]:
# from llama_index.core import PromptTemplate

# react_system_header_str = """\
# You are designed to help with financial inquiries and provide accurate and reliable information. Even if you have prior knowledge about a topic, you must use the tools available to verify or retrieve the most up-to-date information. This is crucial for ensuring that the information you provide is accurate and up-to-date.

# ## Tools
# You have access to a wide variety of tools. You are responsible for using
# the tools in any sequence you deem appropriate to complete the task at hand.

# You have access to the following tools:
# 1. **stock_price_tool**: Fetches live stock prices for various Indian stock symbols. This tool requires the stock symbol as input, which are mentioned within the function for reference and returns the latest price for that stock.
# 2. **internet_search_tool**: Performs internet searches to find relevant information based on a query. This tool allows you to specify the number of results and the type of search.
# 3. **news_tool**: Retrieves the latest business news, with a focus on Indian finance or business news. This tool uses a language and region filter to provide relevant news.


# ## Output Format
# Answer in a very constructive and professional manner.

# """

# react_system_prompt = PromptTemplate(react_system_header_str)

In [8]:
def get_stock_price(stock_symbol: str, api_key = "28bb5e7c25msh8edb18316336786p1db82djsnc22d32c5e79e") -> dict:
    """
    Fetches the live stock price for a given stock symbol from the Live Indian Stock Price API.

    Parameters:
    - stock_symbol (str): The stock symbol for which to fetch the price (e.g., ' 'INFY',  # Infosys
    'TCS',  # Tata Consultancy Services
    'HDFCBANK',  # HDFC Bank
    'RELIANCE',  # Reliance Industries
    'ICICIBANK',  # ICICI Bank
    'HINDUNILVR',  # Hindustan Unilever
    'KOTAKBANK',  # Kotak Mahindra Bank
    'ITC',  # ITC Limited
    'BHARTIARTL',  # Bharti Airtel
    'L&T',  # Larsen & Toubro
    'HDFC',  # Housing Development Finance Corporation
    'SBIN',  # State Bank of India
    'AXISBANK',  # Axis Bank
    'MARUTI',  # Maruti Suzuki
    'TATAMOTORS',  # Tata Motors
    'SUNPHARMA',  # Sun Pharmaceutical Industries
    'WIPRO',  # Wipro Limited
    'ONGC',  # Oil and Natural Gas Corporation
    'TECHM',  # Tech Mahindra
    'ADANIGREEN',  # Adani Green Energy
    'NTPC',  # NTPC Limited
    'M&M',  # Mahindra & Mahindra
    'POWERGRID',  # Power Grid Corporation of India
    'DIVISLAB',  # Divi's Laboratories
    'INDUSINDBK'  # IndusInd Bank).

    Returns:
    - dict: The response from the API containing the stock price information.
    """
    url = f"https://live-indian-stock-price.p.rapidapi.com/api/v1/stock/{stock_symbol}/NSE"
    
    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "live-indian-stock-price.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

stock_price_tool = FunctionTool.from_defaults(fn = get_stock_price)

In [9]:

def bing_news_search(query: str, api_key: str = "1d6de50036de4f71b63896eb7f0b2821", max_results: Optional[int] = 5) -> List[Dict]:
    """
    Search for news using Bing News Search API.

    Args:
        query (str): The query to be passed to Bing News Search.
        api_key (str): The API key for Bing Search.
        max_results (Optional[int]): The maximum number of results to be returned. Defaults to 5.

    Returns:
        List[Dict]: A list of dictionaries containing the news results.
    """
    url = "https://api.bing.microsoft.com/v7.0/news/search"
    headers = {
        "Ocp-Apim-Subscription-Key": api_key
    }
    params = {
        "q": query,
        "count": max_results,
        "freshness": "Day"  
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    results = response.json()
    news_articles = results.get("value", [])
    
    formatted_results = [
        {
            "name": article["name"],
            "url": article["url"],
            "snippet": article["description"]
        }
        for article in news_articles
    ]
    
    return formatted_results

internet_search_tool = FunctionTool.from_defaults(fn=bing_news_search)

In [10]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker
react_agent = ReActAgent.from_tools(tools=[stock_price_tool,internet_search_tool],llm=llm, verbose=True)
# react_agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [11]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'stop', 'bye']:
        print("Goodbye!")
        break

    response = react_agent.chat(user_input)
    print("Bot:", response)

> Running step 139b458f-7389-4b07-a2bd-7497ac2c9e0d. Step input: give me latest news on indian bussiness.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: bing_news_search
Action Input: {'query': 'latest news on Indian business', 'api_key': '1d6de50036de4f71b63896eb7f0b2821', 'max_results': 5}
Observation: [{'name': "'Adopt adequate precaution': India updates trade advisory for enterprises doing business with Chinese firms", 'url': 'https://www.msn.com/en-in/money/news/adopt-protection-indian-embassy-issues-trade-advisory-on-smes-doing-business-with-chinese-firms/ar-BB1qSrzu', 'snippet': 'The Indian Embassy in China issued a new trade advisory for Indian SMEs conducting business with Chinese firms, urging cautious engagement. The adviso'}, {'name': 'LIVE news: 144 dead in Wayanad landslides, many still trapped as rescue ops continue', 'url': 'https://www.business-standard.com/india-news/live-news-today-breaking-news-upd

ValueError: Reached max iterations.